In [127]:
import pandas as pd
import os
import re
import numpy as np
from os import listdir
from tqdm import tqdm

In [3]:
master_policies = pd.read_csv('../data/raw/COVID19_policies - Camila.csv')
inafed_data = pd.read_excel('../data/raw/inafed_bd_webs_subset-Cam.xlsx')

In [6]:
master_policies.head()

,code_INEGI,nom_ent,nom_mun,type,date,end_date,source1,source2,source_type1,source_type2,nom_ent_upper,nom_mun_upper,missing
0,1004,Aguascalientes,Cosío,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGUASCALIENTES,COSIO,1
1,1008,Aguascalientes,San José de Gracia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AGUASCALIENTES,SAN JOSE DE GRACIA,1
2,2006,Baja California,San Quintín,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAJA CALIFORNIA,SAN QUINTIN,1
3,3003,Baja California Sur,La Paz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAJA CALIFORNIA SUR,LA PAZ,1
4,3008,Baja California Sur,Los Cabos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAJA CALIFORNIA SUR,LOS CABOS,1


In [7]:
inafed_data.head()

,estado,municipio,id_estado,id_municipio,cve_inegi,nombre,ap_paterno,ap_materno,sexo,partido,...,direccion,pag_web,cve_lada,telefono,links_websites2,link_to_check,social_media_link1,recover_data1,social_media_link2,local_account
0,Aguascalientes,CosÌo,1,4,1004,EUSEBIO ENRIQUE,DELGADO,ESPARZA,H,PVEM,...,"PRESIDENCIA MUNICIPAL PLAZA JUAREZ S/N, ZONA C...",http://www.cosioaguascalientes.gob.mx/,458,9-87-09-40; 9-87-13-42 EXT. 1,http://www.cosioaguascalientes.gob.mx/,1,https://www.facebook.com/OficialGobiernodeCosio/,1.0,NaN,NaN
1,Aguascalientes,San JosÈ de Gracia,1,8,1008,MARIA CRISTINA,LOPEZ,GONZALEZ,M,PAN,...,"PRESIDENCIA MUNICIPAL JUAN DOMINGUEZ # 101, CE...",https://sanjosedegracia.gob.mx/h-ayuntamiento/,465,9-67-50-92; 9-67-35-32,https://sanjosedegracia.gob.mx/h-ayuntamiento/,1,NaN,NaN,NaN,NaN
2,Baja California,San QuintÌn,2,6,2006,--,--,--,--,--,...,--,--,--,--,--,0,NaN,NaN,NaN,NaN
3,Baja California Sur,La Paz,3,3,3003,RUBEN GREGORIO,MU?OZ,ALVAREZ,H,COAL.,...,"PRESIDENCIA MUNICIPAL LUIS DONALDO COLOSIO, CO...",http://www.lapaz.gob.mx,612,1-23-79-00 EXT. 1100,http://www.lapaz.gob.mx,0,https://www.facebook.com/XVIAyuntamientoLaPaz/,1.0,https://twitter.com/HXVIAyto_LaPaz,NaN
4,Baja California Sur,Los Cabos,3,8,3008,JESUS ARMIDA,CASTRO,GUZMAN,M,COAL.,...,PRESIDENCIA MUNICIPAL BLVD. MIJARES 1413 SAN J...,http://www.loscabos.gob.mx,624,1-46-76-00.; 1-42-00-63,http://www.loscabos.gob.mx,0,https://www.facebook.com/ayuntamientodeloscabos,1.0,NaN,NaN


In [19]:
links_list = []
socialmedia1 = []
socialmedia2 = []

for i in tqdm(range(0, len(inafed_data))):
    cve = inafed_data.loc[i]['cve_inegi']
    if cve in list(master_policies.code_INEGI):
        link_website = inafed_data.loc[i]['links_websites2']
        social1, social2 = inafed_data.loc[i]['social_media_link1'], inafed_data.loc[i]['social_media_link2']
        links_list.append(link_website)
        socialmedia1.append(social1)
        socialmedia2.append(social2)
    else:
        pass

100%|██████████| 1768/1768 [00:01<00:00, 1038.76it/s]


In [20]:
master_policies['link_website'] = links_list
master_policies['social_media1'] = socialmedia1
master_policies['social_media2'] = socialmedia2

In [39]:
social_media_all = []
for i in socialmedia1:
    if pd.isna(i):
        pass
    else:
        if i not in social_media_all:
            social_media_all.append(i.lower())
        else:
            pass
for i in socialmedia2:
    if pd.isna(i):
        pass
    else:
        if i not in social_media_all:
            social_media_all.append(i.lower())
        else:
            pass

In [24]:
len(social_media_all)

434

In [114]:
path_accounts = '../data/facebook_local-gov/'

mapping = {}
for i in listdir(path_accounts):
    
    name = re.sub('[.]xlsx' ,'', i).lower() #.split('.')[0]
    
    if len(name) == 1:
        pass
    else:
        for j in social_media_all:
            if pd.isna(j):
                pass
            else:
                if any(re.findall(name, j)):
                    mapping[j] = i
                    

In [125]:
def assign_socialmedia_account(x, dict_):
    if pd.isna(x):
        return(np.nan)
    else:
        if x in dict_.keys():
            return(dict_[x])
        else:
            return(np.nan)

In [128]:
master_policies['account_social1'] = master_policies.apply(lambda x: assign_socialmedia_account(x['social_media1'], mapping), axis = 1)
master_policies['account_social2'] = master_policies.apply(lambda x: assign_socialmedia_account(x['social_media2'], mapping), axis = 1)

In [130]:
master_policies.to_excel('../data/raw/COVID19_policies - Camila_Social-Media.xlsx')

In [116]:

for i in listdir(path_accounts):
    if i not in mapping.values():
        

3003.csv
Municipio de San Bartolo Coyotepec 2020-2022.xlsx
Ayuntamiento de Atoyatempan 2018-2021.xlsx
Honorable Ayuntamiento de Chocamán 2018 -2021.xlsx
AdministracionMunicipalZimapan.xlsx
Santa Ana Maya 2018 - 2021.xlsx
arturocruzgarcia69.xlsx
Municipio De Soyopa 2018-2021.xlsx
Heroico Xochiapulco.xlsx
1004.csv
Téul de Gonzaléz Ort administración 2018-2021.xlsx
GobiernoJCB.xlsx
Ayuntamiento de Mezquitic Jalisco 2018-2021.xlsx
VP1821.xlsx
Poanas Gobierno 2019-2022.xlsx
H. Ayuntamiento Constitucional de Texistepec, ver. 2018-2021.xlsx
H. Ayuntamiento Constitucional de Teotlalco 2018-2021.xlsx
GobTamazula.xlsx
Ayuntamiento Tolcayuca 2020 - 2024.xlsx
H. Ayuntamiento Huehuetlan El Grande 2018-2021.xlsx
AyuntamientoTasquillo.xlsx
Zapotitlan de Vadillo Oficial Ayuntamiento 2018-2021.xlsx
H. Ayuntamiento Ixhuatlán De Madero 2018-2021.xlsx
Ayuntamiento Tonaya Jalisco.xlsx
H. Ayuntamiento de Samahil 2018-2021.xlsx
Ayuntamiento Tahmek 2018-2021.xlsx
Ayuntamiento Santa Inés Ahuatempan, Puebla.xls

In [120]:
social_media_all

['https://www.facebook.com/oficialgobiernodecosio/',
 'https://www.facebook.com/xviayuntamientolapaz/',
 'https://www.facebook.com/ayuntamientodeloscabos',
 'https://www.facebook.com/ayuntamientodecampeche/',
 'https://www.facebook.com/municipio-de-abasolo-gto-258493014860485/',
 'https://www.facebook.com/municipioixtlahuacan/',
 'https://www.facebook.com/ayuntamiento.tuxtlagutierrez',
 'https://www.facebook.com/gobiernoamigoamatenango',
 'https://www.facebook.com/h.ayuntamientohuitiupan/',
 'https://www.facebook.com/ayuntamiento-de-jitotol-chiapas-122644148369182/',
 'https://www.facebook.com/turismolarrainzar-603446366770820/',
 'https://www.facebook.com/metapa2018/',
 'https://www.facebook.com/h.ayuntamientoocosingo2021/',
 'https://www.facebook.com/ayuntamiento.ocotepec.7',
 'https://www.facebook.com/ayuntamiento.sabanilla.1',
 'https://www.facebook.com/solosuchiapaoficial/',
 'http://www.facebook.com/sunuapachiapa',
 'https://www.facebook.com/gobiernodetapachula/',
 'https://www.f

In [104]:
i = listdir(path_accounts)[2]

In [106]:
name = re.sub('[.]xlsx' ,'', i).lower().split('.')[1]

IndexError: list index out of range

In [96]:
    for name in social_media_all:
        if pd.isna(name):
            pass
        else:
            if any(re.findall(name, i)):
                print(name)
                mapping[i] = name

In [97]:
re.findall(name, i)

[]

In [44]:
social_media_all

['https://www.facebook.com/oficialgobiernodecosio/',
 'https://www.facebook.com/xviayuntamientolapaz/',
 'https://www.facebook.com/ayuntamientodeloscabos',
 'https://www.facebook.com/ayuntamientodecampeche/',
 'https://www.facebook.com/municipio-de-abasolo-gto-258493014860485/',
 'https://www.facebook.com/municipioixtlahuacan/',
 'https://www.facebook.com/ayuntamiento.tuxtlagutierrez',
 'https://www.facebook.com/gobiernoamigoamatenango',
 'https://www.facebook.com/h.ayuntamientohuitiupan/',
 'https://www.facebook.com/ayuntamiento-de-jitotol-chiapas-122644148369182/',
 'https://www.facebook.com/turismolarrainzar-603446366770820/',
 'https://www.facebook.com/metapa2018/',
 'https://www.facebook.com/h.ayuntamientoocosingo2021/',
 'https://www.facebook.com/ayuntamiento.ocotepec.7',
 'https://www.facebook.com/ayuntamiento.sabanilla.1',
 'https://www.facebook.com/solosuchiapaoficial/',
 'http://www.facebook.com/sunuapachiapa',
 'https://www.facebook.com/gobiernodetapachula/',
 'https://www.f

In [43]:
name

'zacapala.go'